In [1]:
from helpers.models import Models
from helpers.llm_client import LLMClient
from helpers.parsers import license_parser
from helpers.functions import *
import pandas as pd

/home/jimbo/Desktop/GSoC24/gsoc24env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# First, using Sentence Transformers

In [2]:
create_license_dataset('extras/license_information/details')
client = LLMClient()
df = client.temp_function(pd.read_csv('extras/lamma3-8b-pytorch-main-sampled.csv'))
file_idx = 0
results = get_top_similar_license_lines(\
                df.loc[file_idx, 'file_comments'],
                'extras/license_information/license_dataset.txt',
                # model='bow',
                top_k=5,
                embedding_approach='license-embedding'
            )
results

License dataset file created successfully at extras/license_information/license_dataset.txt


/home/jimbo/Desktop/GSoC24/gsoc24env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pre-embedded licenses from: extras/license_information/license_embeddings/768_SentenceTransformer-license-embedding.pkl


[(40,
  0.6339692,
  ' File distributed under the Zero Clause BSD (0BSD) license. Copyright Contributors to the pythoncapi_compat project.',
  'License Name: CNRI Python License'),
 (0,
  0.5076868,
  'Header file providing new C API functions to old Python versions.',
  'N/A'),
 (1,
  0.5062,
  'SPDX-License-Identifier: 0BSD',
  'License Name: Xdebug License v 1.03'),
 (41,
  0.4910386,
  ' Homepage: https://github.com/python/pythoncapi_compat',
  'License Name: CNRI Python License'),
 (46,
  0.47866815,
  ' bpo-43753 added Py_Is(), Py_IsNone(), Py_IsTrue() and Py_IsFalse() to Python 3.10.0b1.',
  'License Name: CNRI Python License')]

In [9]:
idx = []
for r in results:
    idx.append(r[0])

In [10]:
def extract_areas(lines, indices, context_lines=3):
    """Extracts text areas around specified indices in a file, returning start and end indices.

    Args:
        file_path: Path to the text file.
        indices: A list of 0-based line indices (integers) to extract around.
        context_lines: Number of lines to include above and below each index.

    Returns:
        A list of tuples, each of the form (start_index, end_index) representing 
        the starting and ending line indices (inclusive) of an extracted area.
    """

    areas = []

    # Sort and deduplicate indices
    sorted_indices = sorted(list(set(indices)))

    for index in sorted_indices:
        start_line = max(0, index - context_lines)
        end_line = min(len(lines) - 1, index + context_lines)  # Inclusive end index

        # Handle overlaps: Skip extraction if the area is already included
        if not any(start_line <= existing_start <= end_line for existing_start, existing_end in areas):
            areas.append((start_line, end_line))

    extracted_text = ""
    for start, end in areas:
        extracted_text += "\n".join(lines[start:end + 1]) + "\n\n"  # Add newline for separation
    
    return extracted_text.strip()  # Remove trailing newlines

print(extract_areas(df.loc[file_idx, 'file_comments'].split('\n'), idx))

Header file providing new C API functions to old Python versions.
SPDX-License-Identifier: 0BSD
PyFrameObject, PyFrame_GetBack()
Cast argument to PyObject* type.

gh-105922 added PyObject_Vectorcall() to Python 3.9.0a4
bpo-36974 added _PyObject_Vectorcall() to Python 3.8.0b1
PYTHONCAPI_COMPAT
 File distributed under the Zero Clause BSD (0BSD) license. Copyright Contributors to the pythoncapi_compat project.
 Homepage: https://github.com/python/pythoncapi_compat
 Latest version: https://raw.githubusercontent.com/python/pythoncapi_compat/master/pythoncapi_compat.h
 Compatibility with Visual Studio 2013 and older which don't support the inline keyword in C (only in C++): use __inline instead.

bpo-36974 added _PyObject_Vectorcall() to Python 3.8.0b1
PYTHONCAPI_COMPAT
 File distributed under the Zero Clause BSD (0BSD) license. Copyright Contributors to the pythoncapi_compat project.
 Homepage: https://github.com/python/pythoncapi_compat
 Latest version: https://raw.githubusercontent.com/py

In [5]:
print(df.loc[file_idx, 'file_comments'])

Header file providing new C API functions to old Python versions.
SPDX-License-Identifier: 0BSD
PyFrameObject, PyFrame_GetBack()
Cast argument to PyObject* type.
bpo-42262 added Py_NewRef() to Python 3.10.0a3
bpo-42262 added Py_XNewRef() to Python 3.10.0a3
bpo-39573 added Py_SET_REFCNT() to Python 3.9.0a4
bpo-39573 added Py_SET_TYPE() to Python 3.9.0a4
bpo-39573 added Py_SET_SIZE() to Python 3.9.0a4
bpo-40421 added PyFrame_GetCode() to Python 3.9.0b1
bpo-40421 added PyFrame_GetBack() to Python 3.9.0b1
bpo-40421 added PyFrame_GetLocals() to Python 3.11.0a7
bpo-40421 added PyFrame_GetGlobals() to Python 3.11.0a7
bpo-40421 added PyFrame_GetBuiltins() to Python 3.11.0a7
bpo-40421 added PyFrame_GetLasti() to Python 3.11.0b1
gh-91248 added PyFrame_GetVar() to Python 3.12.0a2
gh-91248 added PyFrame_GetVarString() to Python 3.12.0a2
bpo-39947 added PyThreadState_GetInterpreter() to Python 3.9.0a5
bpo-40429 added PyThreadState_GetFrame() to Python 3.9.0b1
bpo-39947 added PyInterpreterState_Get(

# Second, Using TF-IDF